In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml.html
from urllib.parse import urldefrag

In [2]:
title=[]
availability=[]
price=[]
url=[]
ups=[]
category=[]

for page in range(1,51):
  html_text=requests.get(f"https://books.toscrape.com/catalogue/page-{page}.html").content
  soup=BeautifulSoup(html_text, 'lxml')

  #part1
  part1=soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')
  for p in part1:
    for a in p.find_all('a'):
      name=a
    title.append(name['title'])
    availability.append(p.find('p', class_='instock availability').text.replace('\n','').replace(' ',''))
    price.append(p.find('p', class_='price_color').text)

  #part2
  part2=soup.find_all('article')
  for p in part2:
    for a in p.find_all('a', href=True):
      link=a['href']
    url.append('https://books.toscrape.com/catalogue/'+link) 
  

#part3
for u in url:  
  html_text=requests.get(u).content
  soup_new=BeautifulSoup(html_text, 'lxml')
  ups.append(soup_new.find('td').text)
  category.append(soup_new.find_all('li')[2].text.replace('\n',''))

In [3]:
df=pd.DataFrame({
    'title':title,
    'availability':availability,
    'price':price,
    'url':url,
    'ups':ups,
    'category':category
})

df.head(5)

,title,availability,price,url,ups,category
0,A Light in the Attic,Instock,£51.77,https://books.toscrape.com/catalogue/a-light-i...,a897fe39b1053632,Poetry
1,Tipping the Velvet,Instock,£53.74,https://books.toscrape.com/catalogue/tipping-t...,90fa61229261140a,Historical Fiction
2,Soumission,Instock,£50.10,https://books.toscrape.com/catalogue/soumissio...,6957f44c3847a760,Fiction
3,Sharp Objects,Instock,£47.82,https://books.toscrape.com/catalogue/sharp-obj...,e00eb4fd7b871a48,Mystery
4,Sapiens: A Brief History of Humankind,Instock,£54.23,https://books.toscrape.com/catalogue/sapiens-a...,4165285e1663650f,History


In [6]:
df.to_csv('books_scraped.csv')